In [1]:
import os
import pandas as pd
import itertools
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import BertModel, BertTokenizer, BertForSequenceClassification

from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


# 各種パスの設定
######################################################################
base_path = '/workspace' # ベースとなるパスを指定してください。#######
######################################################################
os.makedirs(os.path.join(base_path,'model'), exist_ok=True)  # 学習済みモデルの保存するディレクトリを作成
os.makedirs(os.path.join(base_path,'output'), exist_ok=True)  # 提出用ファイルを出力するディレクトリを作成
train_data_path = os.path.join(base_path,'data/train_data.csv') # 訓練データのパスを指定
test_data_path = os.path.join(base_path,'data/test_data.csv') # テストデータのパスを指定
submit_data_path = os.path.join(base_path,'data/submission.csv') # 提出用サンプルfileのパスを指定
model_file_path = os.path.join(base_path,'model') # 学習済みモデルのパスを指定
output_file_path = os.path.join(base_path,'output/20230303_submission.csv') # 提出用ファイルのパスを指定

In [2]:
class CFG:
    wandb = False
    apex = True
    model = 'microsoft/deberta-v3-large'
    seed = 42
    n_splits = 5
    max_len = 512
    dropout = 0.2
    target_size=4
    n_accumulate=1
    print_freq = 50
    min_lr=1e-6
    scheduler = 'cosine'
    batch_size = 16
    num_workers = 2
    lr = 3e-5
    weigth_decay = 0.01
    epochs = 10
    n_fold = 5
    trn_fold = [0, 1, 2, 3, 4]
    train = True 
    num_warmup_steps = 0
    num_cycles=0.5
    debug = False
    debug_ver2 = False
    gradient_checkpointing = True
    freezing = True

In [3]:
INPUT_DIR = '/workspace/data/'
OUTPUT_DIR = '/workspace/data/output/'
OUTPUT_SUB_DIR = os.path.join(OUTPUT_DIR,'Submission')
OUTPUT_MODEL_DIR = os.path.join('/workspace/model',CFG.model.replace('/','-'))

In [4]:
#　データセットを作成するクラスを定義します。
class PaperDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        title = self.data.iloc[index]['title']
        abstract = self.data.iloc[index]['abstract']
        text = title + ' ' + abstract
        labels = self.data.iloc[index]['y']
        
        # inputsの大きさをmodelが扱えるmax_length(512)に指定
        inputs = self.tokenizer(text, max_length=512, truncation=True)
       
        # print(len(inputs['input_ids']))
        
        return {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            'targets': torch.tensor(labels, dtype=torch.float)
        }

In [5]:
# datasetで定義されたバッチ形式の__getitem__を処理して、オリジナルのtorch.Tensorにする関数
def collate_fn(batch):
    input_ids = pad_sequence([torch.tensor(item["input_ids"]) for item in batch], batch_first=True)
    attention_mask = pad_sequence([torch.tensor(item["attention_mask"]) for item in batch], batch_first=True)
    # labels_0 = ([item['targets'][0] for item in batch])
    # labels_1 = ([item['targets'][1] for item in batch])
    # labels = torch.tensor([labels_0, labels_1])
    labels = torch.tensor([item['targets'] for item in batch])


    return {"input_ids": input_ids, "attention_mask": attention_mask, 'targets': labels}


In [6]:
# 各種データの読込
train_df = pd.read_csv(train_data_path) # 訓練データの読込
test_df = pd.read_csv(test_data_path) # テストデータの読込
test_df['y'] = 0 # テストデータのＹの値を初期化
submit_df = pd.read_csv(submit_data_path) # 提出用散布リファイルの読込

In [7]:
skf = StratifiedKFold(n_splits=CFG.n_splits,shuffle=True,random_state=CFG.seed)
for fold, ( _, val_) in enumerate(skf.split(train_df, train_df.y)):
    train_df.loc[val_ , "kfold"] = int(fold)
    
train_df["kfold"] = train_df["kfold"].astype(int)

In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig

#使用する学習済みモデル

#model_checkpoint = "distilbert-base-uncased"
model_checkpoint = "microsoft/deberta-v3-base"
config = AutoConfig.from_pretrained(model_checkpoint)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, config = config)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a Be

In [9]:
#criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

In [10]:
def softmax(z):
    assert len(z.shape) == 2
    s = np.max(z, axis=1)
    s = s[:, np.newaxis] # necessary step to do broadcasting
    e_x = np.exp(z - s)
    div = np.sum(e_x, axis=1)
    div = div[:, np.newaxis] # dito
    return e_x / div

def get_score(outputs, labels):
    outputs = F.softmax(torch.tensor(outputs)).numpy()
    return accuracy_score(np.argmax(outputs,axis=1),labels)

In [11]:
def train(model, dataloader, optimizer):
    model.train()
    train_loss = 0
    for batch in dataloader:
        
        #分類問題なので、目的変数の型をfloatからintに変更
        batch['targets'] = batch['targets'].type(torch.LongTensor)

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        #targets = batch['targets'].view(-1, 1).to(device)

        # debaerta用
        targets = batch['targets'].to(device)
        train_correct = 0

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, labels=targets)

        # 損失の値
        loss = outputs.loss
        # それぞれのラベルの確率
        logits = outputs.logits
        # 確率の大きい方を取る
        _pred= logits.argmax(-1)
        pred = _pred.argmax(-1)

        # ラベルとpredが一致しているものの数だけtrain_correctを加算
        train_correct += pred.eq(targets).sum() 

        # if labels is not None:
        #     loss = nn.functional.binary_cross_entropy_with_logits(logits, labels) * labels.shape[1]
        # train_total += len(pred)
        
        #train_acc = train_correct/8 #正解率
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        torch.cuda.empty_cache()

    return train_loss / len(dataloader)

In [12]:
def validate(model, dataloader, criterion):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            batch['targets'] = batch['targets'].type(torch.LongTensor)

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            #targets = batch['targets'].view(-1, 1).to(device)

            # debaerta用
            targets = batch['targets'].to(device)

            outputs = model(input_ids, attention_mask, labels=targets)

            loss = outputs.loss 

            val_loss += loss.item()

    return val_loss / len(dataloader)


In [13]:
def predict(model, dataloader,val_df):
    outputs = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            # モデルによる予測
            output = model(input_ids, attention_mask=attention_mask)
            logits = output.logits
            # 確率の大きい方を取る
            _pred= F.softmax(logits.to('cpu'))
            #pred = _pred.argmax(-1)
            outputs.append(_pred.to('cpu').detach().numpy())
        outputs = np.concatenate(outputs)[:,0]
        oof_df = val_df.copy()
        oof_df['0'] = list(outputs[:,0])
        oof_df['1'] = list(outputs[:,1])
        torch.cuda.empty_cache()
    return outputs,oof_df

In [ ]:
acc_ave = []
oof_df = pd.DataFrame()
for fold in train_df["kfold"].unique():
    train_df = train_df[train_df["kfold"] != fold]
    val_df = train_df[train_df["kfold"] == fold]
    train_df = train_df.reset_index()
    val_df = val_df.reset_index()
    train_dataset = PaperDataset(train_df, tokenizer)
    val_dataset = PaperDataset(val_df, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=collate_fn)
    
    
    for epoch in tqdm(range(1)):
        print(epoch)
        train_loss = train(model, train_loader, optimizer)
        val_loss = validate(model, val_loader, optimizer)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), os.path.join(OUTPUT_MODEL_DIR,f"{fold}_model.pth")) # ベストなモデルを保存する
        print(f"{fold}_fold")
    
        model.load_state_dict(torch.load(os.path.join(OUTPUT_MODEL_DIR,f"{fold}_model.pth"))) #モデルのロード
        outputs,_oof_df = predict(model, val_loader,val_df)
        oof_df = pd.concat([oof_df, _oof_df])
        acc = get_score(outputs,val_df["y"])
        acc_ave.append(acc)
        print(f"acc:{acc}")
        print(f'Epoch {epoch+1} - train loss: {train_loss:.3f}, val loss: {val_loss:.3f}')
oof_df.to_csv(OUTPUT_MODEL_DIR+f'oof_df.csv', index=False)

In [14]:
fold = 0
val_train_df = train_df[train_df["kfold"] != fold].reset_index()
val_df = train_df[train_df["kfold"] == fold].reset_index()
train_dataset = PaperDataset(val_train_df, tokenizer)
val_dataset = PaperDataset(val_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=collate_fn)


for epoch in tqdm(range(1)):
    print(epoch)
    train_loss = train(model, train_loader, optimizer)
    val_loss = validate(model, val_loader, optimizer)

    # if val_loss < best_val_loss:
    #     best_val_loss = val_loss
    torch.save(model.state_dict(), os.path.join(OUTPUT_MODEL_DIR,f"{fold}_model.pth")) # ベストなモデルを保存する
    print(f"{fold}_fold")

    model.load_state_dict(torch.load(os.path.join(OUTPUT_MODEL_DIR,f"{fold}_model.pth"))) #モデルのロード
    outputs,_oof_df = predict(model, val_loader,val_df)
    oof_df = pd.concat([oof_df, _oof_df])
    acc = get_score(outputs,val_df["y"])
    acc_ave.append(acc)
    print(f"acc:{acc}")
    print(f'Epoch {epoch+1} - train loss: {train_loss:.3f}, val loss: {val_loss:.3f}')
oof_df.to_csv(OUTPUT_MODEL_DIR+f'oof_df.csv', index=False)

  0%|          | 0/1 [00:00<?, ?it/s]

0
0_fold


  0%|          | 0/1 [03:40<?, ?it/s]


TypeError: list indices must be integers or slices, not tuple

In [25]:
outputs = []
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        # モデルによる予測
        output = model(input_ids, attention_mask=attention_mask)
        logits = output.logits
        # 確率の大きい方を取る
        _pred= F.softmax(logits.to('cpu'))
        #pred = _pred.argmax(-1)
        outputs.append(_pred.to('cpu').detach().numpy())